Module 1: Distributed Data Ingestion & Exploration

1. Install and Set Up PySpark

In [1]:
!pip install pyspark


2. Import Required Libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, when, isnan, count


3. Initialize Spark Session

In [3]:
spark = SparkSession.builder \
    .appName("PalatePatterns_Module1") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


4. Load the Amazon Fine Food Reviews Dataset

In [4]:
df = spark.read.csv("/content/Reviews.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5, truncate=False)


root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)

+---+----------+--------------+-----------------------------------+--------------------+----------------------+-----+----------+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |ProductId |UserId        |ProfileNam

5. Clean Data and Handle Missing Values

In [5]:
# Keep rows where 'Text' and 'Score' are present and valid
df_clean = df.filter((col("Text").isNotNull()) & (col("Score").isNotNull()))
df_clean = df_clean.filter((col("Score").cast("int").between(1, 5)))
df_clean = df_clean.withColumn("Score", col("Score").cast("int"))


6. Simulate MapReduce: Review Count by Score

In [6]:
# Mimic MapReduce 'Reduce' step: Aggregation
score_count = df_clean.groupBy("Score").count().orderBy("Score")
score_count.show()


+-----+------+
|Score| count|
+-----+------+
|    1| 52635|
|    2| 29877|
|    3| 42502|
|    4| 80141|
|    5|361648|
+-----+------+



In [7]:
# Save ratings distribution
score_count.toPandas().to_csv("/content/score_distribution.csv", index=False)


7. Add Preprocessing Feature: Review Length

In [8]:
df_clean = df_clean.withColumn("ReviewLength", length(col("Text")))
df_clean.select("Score", "ReviewLength").show(5)


+-----+------------+
|Score|ReviewLength|
+-----+------------+
|    5|         263|
|    1|         194|
|    4|         386|
|    2|         219|
|    5|         140|
+-----+------------+
only showing top 5 rows



In [9]:
# Save review length stats
review_length_stats = df_clean.groupBy("Score").agg({"ReviewLength": "avg"}).toPandas()
review_length_stats.to_csv("/content/review_length_stats.csv", index=False)


8. Drop Duplicate Reviews (UserId + Text)

In [10]:
df_clean = df_clean.dropDuplicates(["UserId", "Text"])


9. Save Preprocessed Data as Partitioned Parquet (HDFS-style layout)

In [11]:
# Save for downstream ML modules
df_clean.write.mode("overwrite").partitionBy("Score").parquet("/content/module1_parquet_output")


Module 2: Streaming Preprocessing & Feature Engineering

Step 1: Load Data from Module 1 Output (Parquet)

In [12]:
# Load the preprocessed Parquet data from Module 1
stream_df = spark.read.parquet("/content/module1_parquet_output")
stream_df.select("Score", "Text").show(5)


+-----+--------------------+
|Score|                Text|
+-----+--------------------+
|    5|This will be the ...|
|    5|I drank this on i...|
|    5|Ordered 4 boxes. ...|
|    5|What I loved abou...|
|    5|I received a very...|
+-----+--------------------+
only showing top 5 rows



Step 2: Assign Sentiment Labels

In [13]:
from pyspark.sql.functions import when

stream_df = stream_df.withColumn("Sentiment",
    when(col("Score") >= 4, "Positive")
    .when(col("Score") == 3, "Neutral")
    .otherwise("Negative"))

stream_df.groupBy("Sentiment").count().show()


+---------+------+
|Sentiment| count|
+---------+------+
| Positive|305119|
|  Neutral| 29608|
| Negative| 57349|
+---------+------+



In [14]:
# Save sentiment distribution
stream_df.groupBy("Sentiment").count().toPandas().to_csv("/content/sentiment_distribution.csv", index=False)


Step 3: Tokenize & Clean the Text

In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="Text", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="clean_tokens")

tokenized_df = tokenizer.transform(stream_df)
cleaned_df = remover.transform(tokenized_df).select("clean_tokens", "Sentiment", "Score")
cleaned_df.show(5, truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----+
|clean_tokens                                                                                                                                                                                                                                                                                                                                                                      |Sentiment|Score|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Step 4: Simulate Streaming Batches (e.g., 30,000 rows per batch)

In [16]:
total_rows = cleaned_df.count()
batch_size = 30000

for start in range(0, total_rows, batch_size):
    batch = cleaned_df.limit(start + batch_size).subtract(cleaned_df.limit(start))
    print(f"\n Batch {start//batch_size + 1}")
    batch.groupBy("Sentiment").count().show()



 Batch 1
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29997|
+---------+-----+


 Batch 2
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29999|
+---------+-----+


 Batch 3
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29997|
+---------+-----+


 Batch 4
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29994|
+---------+-----+


 Batch 5
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29991|
+---------+-----+


 Batch 6
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29993|
+---------+-----+


 Batch 7
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29989|
+---------+-----+


 Batch 8
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29981|
+---------+-----+


 Batch 9
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29992|
+---------+-----+


 Batch 10
+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|29997|
+-------

Step 5: Count Frequent Words Using Explode

In [17]:
from pyspark.sql.functions import explode

exploded = cleaned_df.withColumn("word", explode("clean_tokens"))
exploded.groupBy("word").count().orderBy("count", ascending=False).show(20, truncate=False)


+-------+------+
|word   |count |
+-------+------+
|       |608839|
|like   |142952|
|/><br  |133047|
|good   |91473 |
|one    |89051 |
|great  |83573 |
|taste  |79610 |
|love   |75567 |
|product|65969 |
|get    |62824 |
|coffee |62427 |
|really |59348 |
|tea    |57551 |
|flavor |56809 |
|use    |50220 |
|much   |49396 |
|little |49367 |
|it.    |48601 |
|buy    |45001 |
|find   |44825 |
+-------+------+
only showing top 20 rows



Step 6: Use freqItems for Frequent Word Mining

In [18]:
freq_items = exploded.select("word").freqItems(["word"], support=0.02)
freq_items.show(truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|word_freqItems                                                                                                                                                                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[30, good, make, />not, also, tried, love, butter, , quite, cereal, hot, alot, organic, like, boxes, i

Step 7: Jaccard Similarity Between Example Reviews

In [ ]:
from pyspark.ml.feature import HashingTF, MinHashLSH

hashingTF = HashingTF(inputCol="clean_tokens", outputCol="features", numFeatures=1000)
featurized_data = hashingTF.transform(cleaned_df)

mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
lsh_model = mh.fit(featurized_data)

# Self join to get similar pairs
similar_pairs = lsh_model.approxSimilarityJoin(featurized_data, featurized_data, 0.6, distCol="JaccardDistance")
similar_pairs.select("datasetA.clean_tokens", "datasetB.clean_tokens", "JaccardDistance").show(5, truncate=False)

# Sample a smaller fraction for faster similarity join
# small_sample = featurized_data.sample(withReplacement=False, fraction=0.01, seed=42)

# Then perform similarity join on this smaller set
# similar_pairs = lsh_model.approxSimilarityJoin(small_sample, small_sample, 0.6, distCol="JaccardDistance")
# similar_pairs.select("datasetA.clean_tokens", "datasetB.clean_tokens", "JaccardDistance").show(5, truncate=False)



Step 8: Save Output for Next Module

In [19]:
cleaned_df.write.mode("overwrite").parquet("/content/streaming_preprocessed_data")


# Module 3: Scalable Sentiment Classification

Step 1: Load Preprocessed Data

In [20]:
df = spark.read.parquet("/content/streaming_preprocessed_data")
df.select("clean_tokens", "Sentiment", "Score").show(5, truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----+
|clean_tokens                                                                                                                                                                                                                                                                                                                                                                      |Sentiment|Score|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Step 2: Convert Sentiment to Label

In [21]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Sentiment", outputCol="label")
df = indexer.fit(df).transform(df)


Step 3: Split Data

In [22]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)


Step 4: Define Feature Transformers and Model

In [23]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

hashingTF = HashingTF(inputCol="clean_tokens", outputCol="raw_features", numFeatures=10000)
idf = IDF(inputCol="raw_features", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20)


Step 5: Build and Train the Pipeline

In [25]:
pipeline = Pipeline(stages=[hashingTF, idf, lr])
model = pipeline.fit(train_data)


Step 6: Evaluate Model

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 83.32%


In [27]:
# Save evaluation metrics to JSON
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import json

# Get metrics
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
metrics = {
    "accuracy": evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}),
    "f1": evaluator.evaluate(predictions, {evaluator.metricName: "f1"}),
    "precision": evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}),
    "recall": evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}),
}

# Save as JSON
with open("/content/model_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)


# Module 4: Insights & Visualization

Load Cleaned Review Data

In [28]:
df = spark.read.parquet("/content/module1_parquet_output")
df.select("Score", "Text").show(3, truncate=False)


+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Score|Text                                                                                                                                                                                                                                                                                                                                                                                                                 |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------

Step 3: Clean, Tokenize, and Prepare Words

In [29]:
from pyspark.sql.functions import col, lower, regexp_replace, array_distinct
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import split

df = df.withColumn("clean_text", lower(col("Text")))
df = df.withColumn("clean_text", regexp_replace("clean_text", "[^a-zA-Z\s]", ""))

tokenizer = Tokenizer(inputCol="clean_text", outputCol="words_raw")
tokenized = tokenizer.transform(df)

remover = StopWordsRemover(inputCol="words_raw", outputCol="filtered_words_temp")
tokenized = remover.transform(tokenized)

# Remove duplicates inside each transaction (important for FP-Growth)
tokenized = tokenized.withColumn("filtered_words", array_distinct(col("filtered_words_temp")))


In [30]:
from pyspark.sql.functions import col, size, explode, array, lit
from itertools import combinations

# Step 1: Get co-occurring pairs from filtered tokens
token_pairs = tokenized.select("filtered_words") \
    .filter(size("filtered_words") > 1) \
    .rdd.flatMap(lambda row: [tuple(sorted(pair)) for pair in combinations(set(row["filtered_words"]), 2)])

# Step 2: Count occurrences of pairs
pair_counts = token_pairs.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
pair_counts_df = pair_counts.map(lambda x: (x[0][0], x[0][1], x[1])).toDF(["word1", "word2", "intersection"])

# Step 3: Count individual word occurrences
word_counts = tokenized.select(explode("filtered_words").alias("word")) \
    .groupBy("word").count().withColumnRenamed("count", "word_count")

word_counts1 = word_counts.withColumnRenamed("word", "word1").withColumnRenamed("word_count", "count1")
word_counts2 = word_counts.withColumnRenamed("word", "word2").withColumnRenamed("word_count", "count2")

# Step 4: Join and calculate Jaccard
jaccard_df = pair_counts_df \
    .join(word_counts1, on="word1") \
    .join(word_counts2, on="word2") \
    .withColumn("jaccard", col("intersection") / (col("count1") + col("count2") - col("intersection")))

# Step 5: Save top 100 pairs to CSV for Streamlit
jaccard_top = jaccard_df.orderBy(col("jaccard").desc()).limit(100)
jaccard_top.toPandas().to_csv("jaccard_similarity_top.csv", index=False)


Step 4: Apply FP-Growth for Frequent Patterns

In [31]:
from pyspark.ml.fpm import FPGrowth

fp_growth = FPGrowth(itemsCol="filtered_words", minSupport=0.02, minConfidence=0.5)
fp_model = fp_growth.fit(tokenized)

# sampled_data = tokenized.sample(fraction=0.5, seed=42)  # use 10% of data
# fp_model = fp_growth.fit(sampled_data)


# Show frequent itemsets
fp_model.freqItemsets.orderBy("freq", ascending=False)

# Show association rules
fp_model.associationRules.orderBy("confidence", ascending=False)


DataFrame[antecedent: array<string>, consequent: array<string>, confidence: double, lift: double, support: double]

Prepare Data for Graph-Based Analysis (Optional Advanced Step)

In [32]:
from pyspark.sql.functions import explode, col, monotonically_increasing_id, array_distinct

# Step 1: Add unique row_id
tokenized_with_id = tokenized.withColumn("row_id", monotonically_increasing_id())

# Step 2: Explode with row_id to track origin
exploded = tokenized_with_id.select("row_id", explode("filtered_words").alias("word"))

# Step 3: Join on same row_id to create word pairs (excluding self-pairs)
word_pairs = exploded.alias("a").join(
    exploded.alias("b"),
    (col("a.row_id") == col("b.row_id")) & (col("a.word") < col("b.word"))  # avoid a=b and duplicates
).select(
    col("a.word").alias("src"),
    col("b.word").alias("dst")
).distinct()

# Show some sample pairs
# word_pairs.show(10, truncate=False)


Save FP Results

In [33]:
from pyspark.sql.functions import concat_ws

# Convert array columns to string for CSV compatibility
freq_itemsets_df = fp_model.freqItemsets.withColumn("items", concat_ws(", ", "items"))
assoc_rules_df = fp_model.associationRules \
    .withColumn("antecedent", concat_ws(", ", "antecedent")) \
    .withColumn("consequent", concat_ws(", ", "consequent"))

freq_itemsets_df = freq_itemsets_df.repartition(4)  # use 4 partitions
assoc_rules_df = assoc_rules_df.repartition(4)

# Save as CSV (multi-part for efficiency)
freq_itemsets_df.repartition(10).write.option("header", True).mode("overwrite").csv("/content/freq_itemsets_csv")
assoc_rules_df.repartition(10).write.option("header", True).mode("overwrite").csv("/content/assoc_rules_csv")


In [34]:
import os
import shutil

# Define output folder and final merged file name
output_dir = "/content/freq_itemsets_csv"
merged_file = "/content/freq_itemsets_final.csv"

# List all part files
part_files = [f for f in os.listdir(output_dir) if f.startswith("part-") and f.endswith(".csv")]

# Merge into one file
with open(merged_file, 'w') as outfile:
    for idx, fname in enumerate(sorted(part_files)):
        with open(os.path.join(output_dir, fname), 'r') as infile:
            if idx == 0:
                shutil.copyfileobj(infile, outfile)  # keep header for the first file
            else:
                next(infile)  # skip header in remaining files
                shutil.copyfileobj(infile, outfile)

print("CSV saved at:", merged_file)


CSV saved at: /content/freq_itemsets_final.csv


In [35]:
import os
import shutil

# Define output folder and final merged file name
output_dir = "/content/assoc_rules_csv"
merged_file = "/content/assoc_rules_csv_final.csv"

# List all part files
part_files = [f for f in os.listdir(output_dir) if f.startswith("part-") and f.endswith(".csv")]

# Merge into one file
with open(merged_file, 'w') as outfile:
    for idx, fname in enumerate(sorted(part_files)):
        with open(os.path.join(output_dir, fname), 'r') as infile:
            if idx == 0:
                shutil.copyfileobj(infile, outfile)  # keep header for the first file
            else:
                next(infile)  # skip header in remaining files
                shutil.copyfileobj(infile, outfile)

print("CSV saved at:", merged_file)


CSV saved at: /content/assoc_rules_csv_final.csv


PageRank Implementation

In [ ]:
import networkx as nx
import pandas as pd

# Step 1: Convert word pairs to Pandas DataFrame
word_pairs_pd = word_pairs.toPandas()

# Step 2: Create a directed graph
G = nx.DiGraph()
G.add_edges_from(zip(word_pairs_pd['src'], word_pairs_pd['dst']))

# Step 3: Run PageRank
pagerank_scores = nx.pagerank(G, alpha=0.85)

# Step 4: Convert to DataFrame and sort
pagerank_df = pd.DataFrame(pagerank_scores.items(), columns=["word", "pagerank"]).sort_values(by="pagerank", ascending=False)

# Step 5: Save top 100 ranked words for Streamlit
pagerank_df.head(100).to_csv("/content/pagerank_words.csv", index=False)
print("PageRank CSV saved at: /content/pagerank_words.csv")
